In [1]:
# shopping_app.py
# A complete console-based shopping application with ownership and wallet transfers.
# Ownable module included directly for self-containment.

import sys

class Ownable:
    """Simple Ownable base class for owner privileges."""
    def __init__(self):
        self._owner = None

    @property
    def owner(self):
        return self._owner

    @owner.setter
    def owner(self, value):
        self._owner = value

    def transferOwnership(self, new_owner):
        # Simplified: assumes caller is authorized (e.g., current owner).
        # In a full implementation, add caller check if needed.
        self._owner = new_owner

class Wallet:
    def __init__(self, balance=0):
        self.balance = balance

    def transfer(self, to_wallet, amount):
        if self.balance >= amount:
            self.balance -= amount
            to_wallet.balance += amount
            return True
        return False

class Item(Ownable):  # Inherits from Ownable for owner privileges
    def __init__(self, name, price, quantity):
        super().__init__()  # Initializes Ownable's _owner
        self.name = name
        self.price = price
        self.quantity = quantity
        # owner is set via Ownable's property later

class Cart:
    def __init__(self, owner):
        self.owner = owner
        self.items = []  # List of (item, quantity) tuples

    def add_item(self, item, quantity):
        if item.quantity >= quantity:
            self.items.append((item, quantity))
            item.quantity -= quantity
            return True
        return False

    def check_out(self):
        purchased = []
        total = 0
        for item, qty in self.items:
            cost = item.price * qty
            total += cost
            # Transfer price from buyer's wallet to item's owner (store) wallet
            self.owner.wallet.transfer(item.owner.wallet, cost)
            # Transfer ownership to buyer (uses Ownable method)
            item.transferOwnership(self.owner)
            purchased.append((item, qty))
        
        # Empty cart
        self.items = []
        return total, purchased

class Store:
    def __init__(self, name="DIC Store"):
        self.name = name
        self.wallet = Wallet(0)
        self.inventory = []

    def add_item(self, item):
        item.owner = self  # Uses Ownable's setter
        self.inventory.append(item)

    def display_inventory(self):
        print(f"📦 {self.name} Inventory Status")
        print("+------+-----------+-------+------+")
        print("| No. | Product Name | Amount | Quantity |")
        print("+======+===========+=======+======+")
        for i, item in enumerate(self.inventory):
            print(f"| {i:2d} | {item.name:<9} | {item.price:5d} | {item.quantity:4d} |")
        print("+------+-----------+-------+------+")

class User:
    def __init__(self, name):
        self.name = name
        self.wallet = None
        self.property = []  # Items owned by user

def get_int_input(prompt):
    """Helper function to get a valid integer input with error handling."""
    while True:
        try:
            return int(input(prompt).strip())
        except ValueError:
            print("Invalid input! Please enter a valid number.")

def print_table(items, headers, widths):
    print("+".join(["-" * w for w in widths]))
    print("| " + " | ".join([h.ljust(w-2) for h,w in zip(headers, widths)]) + " |")
    print("+".join(["-" * w for w in widths]))
    for i, row in enumerate(items):
        print("| " + " | ".join([str(cell).ljust(w-2) for cell,w in zip(row, widths)]) + " |")
    print("+".join(["-" * w for w in widths]))

def main():
    print("🤖 Please tell me your name.")
    name = input().strip()
    user = User(name)

    print("Please enter the amount you would like to charge to your wallet")
    initial_balance = get_int_input("")
    user.wallet = Wallet(initial_balance)

    store = Store()
    
    # Initialize inventory with sample items (prices and names from transcript)
    items_data = [
        ("2.5 inch SSD", 13370, 10),
        ("3.5 inch HDD", 10980, 10),
        ("CPU", 40830, 10),
        ("CPU Cooler", 13400, 10),
        ("M.2 SSD", 12980, 10),
        ("PC Case", 8727, 10),
        ("Graphics Board", 23800, 10),
        ("Motherboard", 28980, 10),
        ("Memory", 13880, 10),
        ("Power Supply Unit", 8980, 10),
    ]
    for name, price, qty in items_data:
        item = Item(name, price, qty)
        store.add_item(item)

    cart = Cart(user)

    print("🛍️ Start Shopping")

    while True:
        print("📜 Product List")
        store.display_inventory()

        print("⛏ Please enter item number")
        item_no = get_int_input("")
        if item_no < 0 or item_no >= len(store.inventory):
            print("Invalid item number!")
            continue
        item = store.inventory[item_no]

        print("⛏ Please enter the quantity of goods")
        qty = get_int_input("")
        if qty <= 0 or qty > item.quantity:
            print("Invalid quantity!")
            continue

        if cart.add_item(item, qty):
            print("🛒 Contents of cart")
            headers = ["No.", "Product Name", "Amount", "Quantity"]
            widths = [4, 13, 7, 6]
            table_rows = []
            total = 0
            for i, (itm, q) in enumerate(cart.items):
                subtotal = itm.price * q
                total += subtotal
                table_rows.append([i, itm.name, itm.price, q])
            print_table(table_rows, headers, widths)
            print(f"Total amount: {total}")

            print("😭 Would you like to finish shopping? (yes/no)")
            finish = input().strip().lower()
            if finish == "yes":
                print("💸 Do you wish to confirm your purchase? (yes/no)")
                confirm = input().strip().lower()
                if confirm == "yes":
                    total, purchased = cart.check_out()
                    print("୨୧┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈results┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈୨୧")
                    print(f"🛍️ {user.name}'s property")
                    headers = ["No.", "Product Name", "Amount", "Quantity"]
                    widths = [4, 13, 7, 6]
                    prop_rows = []
                    for i, (itm, q) in enumerate(purchased):
                        prop_rows.append([i, itm.name, itm.price, q])
                    print_table(prop_rows, headers, widths)
                    print(f"😱👛 {user.name}'s wallet balance: {user.wallet.balance}")
                    store.display_inventory()
                    print(f"😻👛 {store.name} wallet balance: {store.wallet.balance}")
                    print("🛒 Contents of cart")
                    empty_rows = []
                    print_table(empty_rows, ["No.", "Product Name", "Amount", "Quantity"], [4, 13, 7, 6])
                    print("🌚 total amount: 0")
                    print("🎉 end")
                    break
                else:
                    # If no confirm, continue shopping
                    continue
        else:
            print("Not enough stock!")

if __name__ == "__main__":
    main()

🤖 Please tell me your name.
Please enter the amount you would like to charge to your wallet
🛍️ Start Shopping
📜 Product List
📦 DIC Store Inventory Status
+------+-----------+-------+------+
| No. | Product Name | Amount | Quantity |
+======+===========+=======+======+
|  0 | 2.5 inch SSD | 13370 |   10 |
|  1 | 3.5 inch HDD | 10980 |   10 |
|  2 | CPU       | 40830 |   10 |
|  3 | CPU Cooler | 13400 |   10 |
|  4 | M.2 SSD   | 12980 |   10 |
|  5 | PC Case   |  8727 |   10 |
|  6 | Graphics Board | 23800 |   10 |
|  7 | Motherboard | 28980 |   10 |
|  8 | Memory    | 13880 |   10 |
|  9 | Power Supply Unit |  8980 |   10 |
+------+-----------+-------+------+
⛏ Please enter item number
⛏ Please enter the quantity of goods
🛒 Contents of cart
----+-------------+-------+------
| No. | Product Name | Amount | Quantity |
----+-------------+-------+------
| 0  | CPU         | 40830 | 5    |
----+-------------+-------+------
Total amount: 204150
😭 Would you like to finish shopping? (yes/no)
💸 